In [1]:
import pygame as pg
import pygame.draw as dw
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.sparse import diags
import scipy.sparse as sps
import scipy.sparse.linalg as spla

pygame 2.6.0 (SDL 2.28.4, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
def cn1DRob_V(xx, tt, F, cr, ci, V):
    Nx = len(xx)
    Nt = len(tt)
    u = np.zeros((Nt,Nx))
    u1 = np.zeros((Nt,Nx))
    dx = abs(xx[0]-xx[1])
    dt = abs(tt[0]-tt[1])
    cMat = np.zeros(Nx)
    
    # matriz coeficientes
    A = sps.csr_matrix(diags([-F/2,1+F+0.5j*V*dt/hbar,-F/2], [-1, 0, 1], shape=(Nx, Nx)))
    B = sps.csr_matrix(diags([F/2,1-F-0.5j*V*dt/hbar,F/2], [-1, 0, 1], shape=(Nx, Nx)))
    # cf - dirichlet
    A[0,0]=1
    A[0,1]=0
    A[-1,-1]=1
    A[-1,-2]=0
    
    #ci
    u=ci.copy()
    for i in range(Nt-1):
        u1 = spla.spsolve(A,B.dot(u)-cMat*F)
        # cf - dirichlet pt 2
        u1[0] = cr[0][2]/cr[0][0]
        u1[-1] = cr[1][2]/cr[1][0]
        u = u1.copy()
    return u/np.linalg.norm(u)

# ESDT - Pot Harmonico

In [67]:
def animate():
    L = 1000
    disp = pg.display.set_mode((L,L), pg.RESIZABLE)
    pg.display.set_caption("Teste #1")
    clk = pg.time.Clock()
    FPS = 60

    def cn1DRob_V(xx, tt, F, cr, ci, V):
        Nx = len(xx)
        Nt = len(tt)
        u = np.zeros((Nt,Nx))
        u1 = np.zeros((Nt,Nx))
        dx = abs(xx[0]-xx[1])
        dt = abs(tt[0]-tt[1])
        cMat = np.zeros(Nx)
        
        # matriz coeficientes
        A = sps.csr_matrix(diags([-F/2,1+F+0.5j*V*dt/hbar,-F/2], [-1, 0, 1], shape=(Nx, Nx)))
        B = sps.csr_matrix(diags([F/2,1-F-0.5j*V*dt/hbar,F/2], [-1, 0, 1], shape=(Nx, Nx)))
        # cf - dirichlet
        A[0,0]=1
        A[0,1]=0
        A[-1,-1]=1
        A[-1,-2]=0
        
        #ci
        u=ci.copy()
        for i in range(Nt-1):
            u1 = spla.spsolve(A,B.dot(u)-cMat*F)
            # cf - dirichlet pt 2
            u1[0] = cr[0][2]/cr[0][0]
            u1[-1] = cr[1][2]/cr[1][0]
            u = u1.copy()
        return u/np.linalg.norm(u)
    
    for i in range(len(tf)):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                return None
            if event.type == pg.VIDEORESIZE:
                disp = pg.display.set_mode((event.w, event.h),pg.RESIZABLE)
                L = np.min([event.h, event.w])
                yL = 0.75*L
        disp.fill((0,0,0)) # refresh fundo da pagina

        xx = np.linspace(-10,10,L)
        dx = abs(xx[1]-xx[0])
        ci = np.sin(4*np.pi*xx) 
        # ci = np.exp(-(xx-2)**2*3)
        ci = ci/np.linalg.norm(np.abs(ci))
        hbar = 6.58e-16 # eV s
        me = 9.11e-31 # kg

        V = np.where(xx>0,100000,0)
        # V = np.exp(-xx**2/10)*50
        
        tt = np.linspace(0,tf[i],200)
        dt = tt[1]-tt[0]
        F = 1j*hbar/(2*me)*dt/dx**2
        u = np.real(cn1DRob_V(xx, tt, F, [[1,0,0],[1,0,0]], ci, V))
        
        # print(list(zip(xx,u)))
        xline = (xx - xx[0]) * L/(xx[-1] - xx[0])
        u = u/np.max(u)*L/10 + L/2
        dw.lines(disp, (255,255,255), False, list(zip(xline,u)), 2)
        pg.display.update()
        clk.tick(FPS)
        x3 = pg.surfarray.pixels3d(disp)
        array = np.uint8(x3)
        im = Image.fromarray(array)
        im = im.save(f'./photos/{i:04d}.png', 'PNG')


tf = np.linspace(0, 1e-16, 200)[1:]
animate()
pg.quit()

